In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Codeforces practice

In [4]:
# https://codeforces.com/problemset/problem/1984/B
def large_addition(N: int) -> bool:
    if N < 10:
        return False
    else:
        N_str: str = str(N)
        length = len(N_str)
        for i in range(length):
            if i == 0 and N_str[i] != '1':
                return False
            elif i == length - 1 and N_str[i] == '9':
                return False
            elif i not in (0, length - 1) and N_str[i] == '0':
                return False

        return True

# Testing the solution
assert large_addition(1393938) == True
assert large_addition(200) == False
assert large_addition(10) == True

LC Virtual: 25th June 2024
https://leetcode.com/contest/smarking-algorithm-contest-3/

In [5]:
# Q1
# https://www.youtube.com/watch?v=x8T11yRLIyY
def minMoves(nums: list[int]) -> int:
    """
    Intution: Instead of increasing all n - 1 elements by 1 each time, try to decrement
    the max element to the smallest element. Relative distances between numbers are the same.
    """
    nums.sort()
    result = 0
    for i in range(len(nums) - 1, -1, -1):
        result += nums[i] - nums[0]
    return result

# Testing the solution
assert minMoves([1,2,3,4]) == 6
assert minMoves([1,2,3]) == 3

In [6]:
# Q2
def numberOfBoomerangs(points: list[list[int]]) -> int:
    def distance(pt1: list[int], pt2: list[int]) -> float:
        return math.sqrt(((pt1[0] - pt2[0]) ** 2) + ((pt1[1] - pt2[1]) ** 2))

    boommerangs: int = 0
    N = len(points)
    for i in range(N):
        curr = points[i]
        distances: dict[float, int] = dict()
        for j in range(N):
            if j != i:
                dist = distance(points[i], points[j])
                distances[dist] = distances.get(dist, 0) + 1

        for count in distances.values():
            boommerangs += math.perm(count, 2)

    return boommerangs

# Testing the solution
assert numberOfBoomerangs([[0,0],[1,0],[2,0]]) == 2
assert numberOfBoomerangs([[1,1],[2,2],[3,3]]) == 2
assert numberOfBoomerangs([[1,1]]) == 0

In [7]:
# Q3
def findMinArrowShots(points: list[list[int]]) -> int:
    # Sort by end and merge the start, end
    points.sort(key=lambda x: (x[1], x[0]))

    # Merge the intervals
    merged_intervals: list[tuple[int, int]] = []
    for start, end in points:
        if merged_intervals and merged_intervals[-1][1] >= start:
            merged_intervals[-1] = max(merged_intervals[-1][0], start), min(merged_intervals[-1][1], end)
        else:
            merged_intervals.append((start, end))

    return len(merged_intervals)

# Testing the solution
assert findMinArrowShots([[10,16],[2,8],[1,6],[7,12]]) == 2
assert findMinArrowShots([[1,2],[3,4],[5,6],[7,8]]) == 4
assert findMinArrowShots([[1,2],[2,3],[3,4],[4,5]]) == 2

In [8]:
# Q4
def numberOfArithmeticSlicesBrute(nums: list[int]) -> int:
    def is_valid(sub: list[int]) -> bool:
        if len(sub) < 3:
            return False
        else:
            diff = sub[1] - sub[0]
            for i in range(len(sub) - 1):
                if sub[i + 1] - sub[i] != diff:
                    return False
            return True

    count = 0
    def backtrack(i: int, subseq: list[int]):
        if i == N:
            count += is_valid(subseq)
        else:
            subseq.append(nums[i])
            backtrack(i + 1, subseq)
            subseq.pop()
            backtrack(i + 1, subseq)

    N = len(nums)
    backtrack(0, [])
    return count

In [9]:
# Upsolving
# https://www.youtube.com/watch?v=YIMwwT9JdIE
def numberOfArithmeticSlices(nums: list[int]) -> int:
    """
    Ending at index 'j' what is maximum number of arthimetic subsequences that I can form >= 2
    """
    N = len(nums)
    dp: list[collections.defaultdict[int, int]] = [collections.defaultdict(int) for _ in range(N)]
    sub_count = dp_sum = 0
    for j in range(N):
        for i in range(j):
            delta = nums[i] - nums[j]
            dp[j][delta] += dp[i][delta] + 1
            sub_count, dp_sum = sub_count + 1, dp_sum + dp[i][delta] + 1

    return dp_sum - sub_count

# Testing the solution
assert numberOfArithmeticSlices([2,4,6,8,10]) == 7
assert numberOfArithmeticSlices([7,7,7,7,7]) == 16